In [ ]:
#from gensim.models import Word2Vec 
import numpy as np
import re
import pandas as pd
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import classification_report, confusion_matrix
# https://code.google.com/archive/p/word2vec/
# https://nlp.stanford.edu/projects/glove/

### GloVe (~100MB)

In [ ]:
%%time
glove_model = api.load('glove-twitter-25')


CPU times: total: 30.8 s
Wall time: 31.3 s


In [ ]:
df = pd.read_csv('yelp_labelled.txt', header=None, delimiter='\t', names=['txt', 'sentiment'])
df

,txt,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


# Explore dataset, answer one or more of these questions:
### 1. How many words are in each entry?
### 2. Can you normalize words ending with n't (like hadn't, don't) to had not, do not etc?
### 3. What is the size of each class?
### 4. What is the longest and the shortest entry?
### 5. How many words are known to GloVe in each entry?
### 6. Does the dataset contains duplications? Empty values?
### 7. Assign a good/bad sentiment score to each entry based on the word vectors
### 8. How correlated is [7] with the 0/1 labels?

# Solutions

### 1. How many words are in each entry?

In [4]:
df['num_of_words'] = df['txt'].apply(lambda x : len(x.split(' ')))
df.sample(5)

,txt,sentiment,num_of_words
340,the spaghetti is nothing special whatsoever.,0,6
414,I can take a little bad service but the food s...,0,11
178,"Not a weekly haunt, but definitely a place to ...",1,17
203,Great brunch spot.,1,3
602,They also have the best cheese crisp in town.,1,9


### 2. Can you normalize words ending with n't (like hadn't, don't) to had not, do not etc?

In [22]:
# I used groups to avoid cases as xxx n't xxx and \b for cases as wasn'tttt

print(re.sub(r"(\w+)n't\b", r"\g<1> not", r"n't wasn't wasn'ttttt"))

df['normed_txt'] = df['txt'].apply(lambda x : re.sub(r"(\w+)n't\b", r"\g<1> not", x))

df[df.txt.str.contains("n't")].sample(5)

n't was not wasn'ttttt


,txt,sentiment,num_of_words,normed_txt,known_to_glove,distance_to_good,distance_to_bad,afeka_sentiment
220,The shower area is outside so you can only rin...,0,25,The shower area is outside so you can only rin...,20,0.934690,0.929481,1
466,I don't think I'll be running back to Carly's ...,0,13,I do not think I'll be running back to Carly's...,8,0.911829,0.934835,0
963,After all the rave reviews I couldn't wait to ...,0,13,After all the rave reviews I could not wait to...,8,0.922220,0.932657,0
994,I can't tell you how disappointed I was.,0,8,I ca not tell you how disappointed I was.,4,0.926724,0.920479,1
294,I didn't know pulled pork could be soooo delic...,1,9,I did not know pulled pork could be soooo deli...,6,0.912949,0.933921,0


### 3. What is the size of each class?

In [6]:
df.sentiment.value_counts()

1    500
0    500
Name: sentiment, dtype: int64

### 4. What is the longest and the shortest entry?

In [7]:
# assuming num of words
df.sort_values(by=['num_of_words']).head(1)


,txt,sentiment,num_of_words,normed_txt
165,DELICIOUS!!,1,1,DELICIOUS!!


In [8]:
df.sort_values(by=['num_of_words']).tail(1)

,txt,sentiment,num_of_words,normed_txt
623,a drive thru means you do not want to wait aro...,0,32,a drive thru means you do not want to wait aro...


### 5. How many words are known to GloVe in each entry?

In [9]:
df['known_to_glove'] = df['txt'].apply(lambda x : sum([1 for item in x.split(' ') if glove_model.__contains__(item)]))

In [10]:
df.sample(5)

,txt,sentiment,num_of_words,normed_txt,known_to_glove
50,We ordered the duck rare and it was pink and t...,1,21,We ordered the duck rare and it was pink and t...,19
431,We loved the biscuits!!!,1,4,We loved the biscuits!!!,2
681,Your staff spends more time talking to themsel...,0,10,Your staff spends more time talking to themsel...,9
377,"In an interesting part of town, this place is ...",1,10,"In an interesting part of town, this place is ...",7
479,I LOVED it!,1,3,I LOVED it!,0


### 6. Does the dataset contains duplications? Empty values?

In [11]:
df.duplicated(subset='txt', keep='first').sum()

4

In [12]:
df[df.duplicated(subset='txt', keep=False)]

,txt,sentiment,num_of_words,normed_txt,known_to_glove
334,I love this place.,1,4,I love this place.,2
380,I won't be back.,0,4,I wo not be back.,2
383,The food was terrible.,0,4,The food was terrible.,2
505,I would not recommend this place.,0,6,I would not recommend this place.,4
814,I love this place.,1,4,I love this place.,2
816,The food was terrible.,0,4,The food was terrible.,2
843,I won't be back.,0,4,I wo not be back.,2
846,I would not recommend this place.,0,6,I would not recommend this place.,4


In [13]:
# another way to find the duplicates...
df.txt.value_counts()

I would not recommend this place.                                                                                                         2
I love this place.                                                                                                                        2
I won't be back.                                                                                                                          2
The food was terrible.                                                                                                                    2
Wow... Loved this place.                                                                                                                  1
                                                                                                                                         ..
The burger... I got the "Gold Standard" a $17 burger and was kind of disappointed.                                                        1
OMG, the food was de

In [14]:
# empty texts
sum(df['txt'].apply(len) == 0)

0

### 7. Assign a good/bad sentiment score to each entry based on the word vectors

In [15]:
def sentence_distance_from_word(good_or_bad, sentence):
    # we define sentence disance as the highest similarity of any of the words in the sentecnce with the target word.
    
    # will tokenizing - remover non-letters and lowercase the word
    list_of_word_vectors = [glove_model[item] for item in re.split('\W+', sentence.lower()) if glove_model.__contains__(item)]
    
    if len(list_of_word_vectors) == 0:
        return 0
    
    # return the closest vector
    return max([cosine_similarity(X=[vector], Y=[glove_model[good_or_bad]])[0][0] for vector in list_of_word_vectors])


In [16]:
sentence_distance_from_word('good', 'be best')

0.9234228

In [17]:
%%time
df['distance_to_good'] = df['txt'].apply(lambda x : sentence_distance_from_word('good', x))

df['distance_to_bad'] = df['txt'].apply(lambda x : sentence_distance_from_word('bad', x))

CPU times: total: 4.83 s
Wall time: 4.82 s


In [18]:
df

,txt,sentiment,num_of_words,normed_txt,known_to_glove,distance_to_good,distance_to_bad
0,Wow... Loved this place.,1,4,Wow... Loved this place.,1,0.938756,0.922743
1,Crust is not good.,0,4,Crust is not good.,3,1.000000,0.929481
2,Not tasty and the texture was just nasty.,0,8,Not tasty and the texture was just nasty.,6,0.891364,0.929481
3,Stopped by during the late May bank holiday of...,1,15,Stopped by during the late May bank holiday of...,11,0.935348,0.936768
4,The selection on the menu was great and so wer...,1,12,The selection on the menu was great and so wer...,10,0.937852,0.911714
...,...,...,...,...,...,...,...
995,I think food should have flavor and texture an...,0,12,I think food should have flavor and texture an...,10,0.911711,0.934835
996,Appetite instantly gone.,0,3,Appetite instantly gone.,1,0.814328,0.867360
997,Overall I was not impressed and would not go b...,0,10,Overall I was not impressed and would not go b...,8,0.891364,0.929481
998,"The whole experience was underwhelming, and I ...",0,16,"The whole experience was underwhelming, and I ...",10,0.916969,0.934835


In [19]:
df['afeka_sentiment'] = (df['distance_to_good'] > df['distance_to_bad']).astype(int)

In [20]:
df

,txt,sentiment,num_of_words,normed_txt,known_to_glove,distance_to_good,distance_to_bad,afeka_sentiment
0,Wow... Loved this place.,1,4,Wow... Loved this place.,1,0.938756,0.922743,1
1,Crust is not good.,0,4,Crust is not good.,3,1.000000,0.929481,1
2,Not tasty and the texture was just nasty.,0,8,Not tasty and the texture was just nasty.,6,0.891364,0.929481,0
3,Stopped by during the late May bank holiday of...,1,15,Stopped by during the late May bank holiday of...,11,0.935348,0.936768,0
4,The selection on the menu was great and so wer...,1,12,The selection on the menu was great and so wer...,10,0.937852,0.911714,1
...,...,...,...,...,...,...,...,...
995,I think food should have flavor and texture an...,0,12,I think food should have flavor and texture an...,10,0.911711,0.934835,0
996,Appetite instantly gone.,0,3,Appetite instantly gone.,1,0.814328,0.867360,0
997,Overall I was not impressed and would not go b...,0,10,Overall I was not impressed and would not go b...,8,0.891364,0.929481,0
998,"The whole experience was underwhelming, and I ...",0,16,"The whole experience was underwhelming, and I ...",10,0.916969,0.934835,0


### 8. How correlated is [7] with the 0/1 labels?

In [21]:
print(classification_report(df['sentiment'], df['afeka_sentiment']))

print(confusion_matrix(df['sentiment'], df['afeka_sentiment']))

              precision    recall  f1-score   support

           0       0.66      0.56      0.61       500
           1       0.62      0.72      0.66       500

    accuracy                           0.64      1000
   macro avg       0.64      0.64      0.63      1000
weighted avg       0.64      0.64      0.63      1000

[[279 221]
 [142 358]]


In [ ]:
lst = [['x', 5], ['y', 5], ['g', 5], ['u', 2]]


In [ ]:
dict(zip([x[0] for x in lst], [x[1] for x in lst]))

{'x': 5, 'y': 5, 'g': 5, 'u': 2}